### Метод рунге-кутты 4-го порялка

In [ ]:

def rungeKutta2Step(f, g, x, y, v, xstep):
    k1 = xstep * f(x, y, v)
    l1 = xstep * g(x, y, v)

    k2 = xstep * f(x + xstep / 2, y + k1 / 2, v + l1 / 2)
    l2 = xstep * g(x + xstep / 2, y + k1 / 2, v + l1 / 2)

    nexty = y + (k1 + k2) / 2
    nextv = v + (l1 + l2) / 2
    return nexty, nextv


def rungeKutta2(x0, u0, v0, xstep, xTarget, f, g):
    x, u, v = x0, u0, v0
    result = [(x, u, v)]
    while x <= xTarget:
        u, v = rungeKutta2Step(f, g, x, u, v, xstep)
        result.append((x, u, v))
        x += xstep
    return result


def rungeKutta4Step(f, g, x, u, v, xstep):
    k1 = xstep * f(x, u, v)
    l1 = xstep * g(x, u, v)

    k2 = xstep * (f(x + 0.5 * xstep, u + 0.5 * k1, v + 0.5 * l1))
    l2 = xstep * (g(x + 0.5 * xstep, u + 0.5 * k1, v + 0.5 * l1))

    k3 = xstep * (f(x + 0.5 * xstep, u + 0.5 * k2, v + 0.5 * l2))
    l3 = xstep * (g(x + 0.5 * xstep, u + 0.5 * k2, v + 0.5 * l2))

    k4 = xstep * (f(x + xstep, u + k3, v + l3))
    l4 = xstep * (g(x + xstep, u + k3, v + l3))

    nextu = u + (k1 + 2 * k2 + 2 * k3 + k4) / 6
    nextv = v + (l1 + 2 * l2 + 2 * l3 + l4) / 6
    return nextu, nextv

def rungeKutta4(x0, u0, v0, xstep, xTarget, f, g):
    x, u, v = x0, u0, v0
    result = [(x, u, v)]
    while x <= xTarget:
        u, v = rungeKutta4Step(f, g, x, u, v, xstep)
        result.append((x, u, v))
        x += xstep
    return result

### Интерполяция плотности плазмы

In [13]:
import numpy as np

TempTable = ((2000, 8.200E-03),
    (3000, 2.768E-02),
    (4000, 6.560E-02),
    (5000, 1.281E-01),
    (6000, 2.214E-01),
    (7000, 3.516E-01),
    (8000, 5.248E-01),
    (9000, 7.472E-01),
    (10000, 1.025E+00),
)

# TempTable = ((2000, 1.6),
#     (3000, 5.4),
#     (4000, 1.280E+01),
#     (5000, 2.500E+01),
#     (6000, 4.320E+01),
#     (7000, 6.860E+01),
#     (8000, 1.024E+02),
#     (9000, 1.458E+02),
#     (10000, 2.000E+02),
# )

def getkFunc():
    tarr, karr = zip(*TempTable)
    tarr = [np.log(x) for x in tarr]
    karr = [np.log(x) for x in karr]
    def k(t):
        return np.exp(np.interp([np.log(t)], tarr, karr)[0])
    return k


### Формула планка и параметры 

In [40]:
Tw = 2000
T0 = 10000
R = 0.35
p = 4

c = 3 * 1e10 # Скорость света

def T(r):
    return (Tw - T0) * (r / R)**p + T0

#u_p(r)
def plankFunc(r):
    return 3.084 * 1e-4 / (np.exp(4.799*1e4 / T(r)) - 1)


### Вычисление варьируемого параметра

In [41]:
# u(0) = lambda * u_p(0)
# v(0) = 0
# v(R) = 0.39 * c * u(R)

rungeKuttaEps = 1e-4

k = getkFunc()

def f(x, u, v):
    return (-3) * k(x) * v / c

def g(x, u, v):
    if (x == 0):
        return k(x) * c * (plankFunc(x) - u) / 2
    return k(x) * c * (plankFunc(x) - u) - v / x

def findSolution(lambdaParam, rungeKuttaEps, initStep):
    x0 = 0
    u0 = lambdaParam * plankFunc(0)
    v0 = 0
    xstep = initStep
    xTarget = R
    prevResult = rungeKutta4(x0, u0, v0, xstep, xTarget, f, g)
    prevTargetu = prevResult[-1][1]
    prevTargetv = prevResult[-1][2]
    while True:
        xstep /= 2
        result = rungeKutta4(x0, u0, v0, xstep, xTarget, f, g)
        targetu = result[-1][1]
        targetv = result[-1][2]
        # print(result[-1])
        if abs(prevTargetu - targetu) / targetu  < rungeKuttaEps:
            break
        prevResult = result
        prevTargetu = targetu
        prevTargetv = targetv
    return targetu, targetv

def equationFunc(x, u, v):
    return 0.39 * c * u - v

left = 1e-10
leftResult = findSolution(left, rungeKuttaEps, 1e-2)
right = 1
rightResult = findSolution(right, rungeKuttaEps, 1e-2)

eps = 1e-6

# Метод половинного деления
while abs((right - left) / left) > eps:
    mid = (left + right) / 2
    midResult = findSolution(mid, rungeKuttaEps, 1e-2)
    if equationFunc(mid, *midResult) * equationFunc(left, *leftResult) < 0:
        right = mid
    else:
        left = mid
    print(f"lambda = {mid}, u = {midResult[0]}, v = {midResult[1]}")
print(f"final lambda = {right}, u = {rightResult[0]}, v = {rightResult[1]}")

finalLambda = mid

lambda = 0.50000000005, u = 1.2808447284928962e-06, v = -11.16837486446373
lambda = 0.250000000075, u = 6.404166421196983e-07, v = 16.401969093677824
lambda = 0.1250000000875, u = 3.2020259893309967e-07, v = 30.187141072748577
lambda = 0.06250000009375001, u = 1.6009557733980024e-07, v = 37.07972706228398
lambda = 0.031250000096875, u = 8.004206654315054e-08, v = 40.52602005705166
lambda = 0.0156250000984375, u = 4.001531114482566e-08, v = 42.24916655443552
lambda = 0.007812500099218751, u = 2.000193344566328e-08, v = 43.110739803127444
lambda = 0.0039062500996093754, u = 9.995244596082064e-09, v = 43.541526427473386
lambda = 0.0019531250998046875, u = 4.9919001712914624e-09, v = 43.75691973964638
lambda = 0.0009765625999023438, u = 2.4902279588961597e-09, v = 43.86461639573287
lambda = 0.0014648438498535156, u = 3.74106406509381e-09, v = 43.81076806768964
lambda = 0.0017089844748291016, u = 4.366482118192637e-09, v = 43.78384390366801
lambda = 0.0015869141623413086, u = 4.053773091643

/tmp/ipykernel_235730/2023902246.py:32: RuntimeWarning:

divide by zero encountered in log



### Решение

In [42]:
import plotly.express as px
import plotly.graph_objects as go

print(f"Lambda найдена: {finalLambda}+-{finalLambda*eps}")

rNumber = 100

rstep = R / rNumber
rvalues, uvalues, vvalues = zip(*rungeKutta4(0, finalLambda * plankFunc(0), 0, rstep, R, f, g))
u_pvalues = [plankFunc(r) for r in rvalues]

fig = px.line()
fig.add_trace(go.Scatter(x=rvalues, y=uvalues, name="u"))
fig.add_trace(go.Scatter(x=rvalues, y=u_pvalues, name="u_p"))
fig.show()

fig = px.line()
fig.add_trace(go.Scatter(x=rvalues, y=vvalues, name="F"))
fig.show()

/tmp/ipykernel_235730/2023902246.py:32: RuntimeWarning:

divide by zero encountered in log



Lambda найдена: 0.0014661840230382533+-1.4661840230382533e-09
